# Lecture 03 : Mini-Project

written by SinsuSquid (bgkang) on 27 November 2024

이제 슬슬 배운 내용들을 좀 써먹어봐야겠죠? 걱정 말아요. 실전이라고 해봤자 실상 내가 code를 짜는거지 여러분은 눈으로 따라가면서 cell만 실행시키면 끝이잖아요? ㅎㅎ
<br>\
내용상으로는 `torch_geometric`을 이용해서 어떠한 target을 갖는 GNN model을 만들어봐야 할 타이밍인데, 문제는 이런저런 GNN tutorial을 뒤져보아도 괜찮은 size의 project를 찾지 못하겠다는거죠. 그래서 간단 (하진 않을 것 같긴 한데) 미니 프로젝트를 직접 만들어보기로 했어요. (워, 워, 박수소리때문에 진행이 안되잖아요.) 저-번에 나온적이 있던 [AqSolDB](https://www.kaggle.com/datasets/sorkun/aqsoldb-a-curated-aqueous-solubility-dataset)를 예측하는 model을 다시 한번 만들어볼건데, 이번엔 GNN을 이용한 model을 활용해보기로 하겠습니다. NN를 이용한 model과 GNN을 이용한 model의 성능을 비교해볼 수 있다면 좋겠지만, 그건 내가 아니라 여러분이 직접 비교해봐야 하는 일인 것 같아요 히히.

## Target

AqSolDB에 대해서는 Lecture 01에서 설명한적 있으니까 생략하도록 할게요. 저어기 가서 다시 읽어보고 오세요.

In [ ]:
!kaggle datasets download -d sorkun/aqsoldb-a-curated-aqueous-solubility-dataset
!unzip -u ./aqsoldb-a-curated-aqueous-solubility-dataset.zip
!rm ./aqsoldb-a-curated-aqueous-solubility-dataset.zip

In [ ]:
import pandas as pd
df = pd.read_csv('./curated-solubility-dataset.csv')
print(df.columns)
df.head()

이번 시간에는 좀 더 과감한 짓을 해볼까요? 화합물의 SMILES (Simplified Molecular Input Line Entry System) 정보와 목표가 되는 solubility 만 챙기고 나머지 정보는 다 버려보도록 하죠.

In [ ]:
df = df.loc[:, ['SMILES', 'Solubility']]
df.head()

In [ ]:
import numpy as np
train, validation, test = np.split(df.sample(frac = 1, random_state = 42),
                                   [int(0.8 * len(df)), int(0.9 * len(df))])
print(f"Train : {len(train)} | Validate : {len(validation)} | Test : {len(test)}")

이런 짓을 하는 이유는 SMILES만 가지고 molecular graph를 만들어주는 code가 이미 있기 때문이죠. (Chemomile을 개발할때도 이 code를 수정해서 썼어요.) 이상하게 PyG 홈페이지에서는 code의 출처를 명확하게 밝혀주고 있진 않네요. 아마 [AttentiveFP](https://pubs.acs.org/doi/10.1021/acs.jmedchem.9b00959) (조금 있다 써볼거에요)를 개발하신 분들이 contribute하신게 아닐까 짐작만 할 뿐입니다.
<br><br>
어떻게 굴러가는지는 한번 봐줘야하니까,

In [ ]:
from torch_geometric.utils import from_smiles

SMILES = 'CCO' # my-cute-little-ethanol
data = from_smiles(SMILES) # SMILES만 집어넣으면 그래프가 만들어진다니!!
print(data)
print('data.x : ', data.x)
print('data.edge_index : ', data.edge_index)
print('data.edge_attr : ', data.edge_attr)
print('data.smiles : ', data.smiles)

`data.x`랑 `data.edge_attr`에 들어가있는 정보는 [AttentiveFP](https://pubs.acs.org/doi/10.1021/acs.jmedchem.9b00959) 논문이나 [documentation](https://pytorch-geometric.readthedocs.io/en/2.6.1/_modules/torch_geometric/utils/smiles.html#from_smiles)보고 찾아보던가 하세요. 나는 알고있지만 별로 안중요할 것 같아서 생략할게요.

## `DataLoader`

Lecture 01에서 `DataLoader`가 뭔지 말하고 넘어갔었죠? 오늘 사용하는 `DataLoader`는 PyTorch의 것을 사용하는게 아니라 PyG의 것을 사용할겁니다. 하는 짓은 거의 비슷한데 graph형태의 data에서 사용하기에 더 알맞게 구성했겠죠. 사용법은 아래와 같아요.

In [ ]:
# List Comprehension은 알아서 공부해와요!
train_dataset = [(from_smiles(row['SMILES']), row['Solubility']) for idx, row in train.iterrows()]
validation_dataset = [(from_smiles(row['SMILES']), row['Solubility']) for idx, row in validation.iterrows()]
test_dataset = [(from_smiles(row['SMILES']), row['Solubility']) for idx, row in test.iterrows()]

In [ ]:
from torch_geometric.loader import DataLoader
# 이제 DataLoader를 만들어줄 차례

train_loader = DataLoader(train_dataset, batch_size = 256)
validation_loader = DataLoader(validation_dataset, batch_size = 256)
test_loader = DataLoader(test_dataset, batch_size = 256) # 참 쉽죠?

사실 `torch_geometric`에서 `DataLoader`가 어떤 방식으로 동작하는지 알면 엄청 신기합니다. 하지만 이걸 설명하려면 Lecture를 통째로 할애해도 부족하기 때문에 넘어가도록 할게요.

In [ ]:
# 사용은 이런식으로
for batch, target in test_loader:
    print(batch, target) # 여러개의 graph를 묶어 하나의 batch로 만들어주고있죠?
    break

## A Model : AttentiveFP

당장 저번시간에 `torch_geometric`이 다양한 layer를 제공한다고 말씀드렸었죠. 이번 시간에는 [AttentiveFP](https://pytorch-geometric.readthedocs.io/en/2.6.1/generated/torch_geometric.nn.models.AttentiveFP.html?highlight=attentivefp#torch_geometric.nn.models.AttentiveFP)라는 model (이미 완성된 형태라 model이나 부르나봐요)을 수정 없이 그냥 가져다 써보도록 하겠습니다. 이 model이 어떤식으로 굴러가는지 알고싶으면 Chemomile의 Metod를 읽어보거나 (엣헴!) AttentiveFP가 처음 소개된 [논문](https://pubs.acs.org/doi/10.1021/acs.jmedchem.9b00959)을 읽어보면 좋을 것 같아요.

In [ ]:
import torch
from torch_geometric.nn.models import AttentiveFP
model = AttentiveFP(in_channels = 9,
                    hidden_channels = 64,
                    out_channels = 1,
                    edge_dim = 3,
                    num_layers = 5,
                    num_timesteps = 5,
                    dropout = 0.2)
model

In [ ]:
%%pprint
# 잘 굴러가는지 한번 볼까요?
# 굳이 loader 형태로 집어넣는 이유는, 
# AttentiveFP model의 forward 함수에는 batch - 말 그대로 batch 정보 - 를 필요로 하는데
# 이걸 직접 만들어서 넣어주기가 귀찮기 때문이죠.
for batch, y in test_loader:
    batch.x = batch.x.to(torch.float)
    print(model(batch.x, batch.edge_index, batch.edge_attr, batch.batch))
    break

## Model Training

training function같은거 구성하는법은 Lecture 01의 내용과 거의 동일해요. 그러니까 이부분에 대한 설명은 skip하도록 할게요. (수업 시간에 처음 본다는듯한 표정으로 강의자를 쳐다본다면 매우 언짢아하겠죠? ^^;)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1E-2)
criterion = torch.nn.MSELoss()

In [ ]:
def train(model, loader):
    model.train()
    tot_loss = 0.0
    for batch, y in loader:
        optimizer.zero_grad()

        # 미리 해주면 더 좋긴 한데, 귀찮네요.
        batch.x = batch.x.to(torch.float)
        y = y.reshape(-1, 1)
        
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        tot_loss += loss.item()

    return tot_loss / len(loader)

def validation(model, loader):
    model.eval()
    tot_loss = 0.0
    with torch.no_grad():
        for batch, y in loader:
            batch.x = batch.x.to(torch.float)
            y = y.reshape(-1, 1)
            
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            loss = criterion(out, y)
    
            tot_loss += loss.item()
    
    return tot_loss / len(loader)
    
trn_loss = []; val_loss = [] # Loss Curve도 그려봐야하니까~
def run(epochs): # epoch 수를 이 부분에서 받아오도록 하죠
    for e in range(epochs):
        trn_loss.append(train(model, train_loader))
        val_loss.append(validation(model, validation_loader))

        print(f"Epoch : {e:05d} | Trn. Loss : {trn_loss[-1]:.3f} | Val. Loss : {val_loss[-1]:.3f}")

    print("Training Complete! >:D")

In [ ]:
run(100) # 확실히 NN보다는 좀 오래 걸리죠?

## Model Validation

Lecture 01에서 다 해본거니까, 알죠? 성능 비교는 결과 다 뽑고 나중에 한번에 해볼라니까 좀만 기다리고 있어요.

In [ ]:
with torch.no_grad():
    model.eval()

    test_loss = 0.0
    pred = []; true = []

    for batch, y in test_loader:
        batch.x = batch.x.to(torch.float)
        y = y.reshape(-1, 1)
        
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        test_loss += criterion(out, y)
        pred.append(out.numpy())
        true.append(y.numpy())

test_loss = test_loss.item() / len(test_loader)
pred = np.concatenate(pred).flatten()
true = np.concatenate(true).flatten()

test_loss

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize = (8, 8), dpi = 200)

ax.set_xlabel("Epoch", fontsize = 15)
ax.set_ylabel("Loss", fontsize = 15)
ax.set_title("Loss Curve", fontsize = 20)

ax.plot(range(100), trn_loss, linewidth = 3, label = 'Training Loss')
ax.plot(range(100), val_loss, linewidth = 3, label = 'Validation Loss')
ax.hlines(test_loss, 0, 100, linestyle = '--', color = 'k', label = 'Test Loss')

ax.legend(fontsize = 15)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 8), dpi = 200)

ax.set_xlabel("True Value", fontsize = 15)
ax.set_ylabel("Predicted Value", fontsize = 15)
ax.set_title("Solubility", fontsize = 20)

ax.plot(true, true, linestyle = '--', color = 'k', linewidth = 3)
ax.plot(true, pred, linestyle = 'None', marker = 'o', markersize = 7, alpha = 0.5)

plt.show()

In [ ]:
mae = np.abs(true - pred).mean()
rmse = np.sqrt(np.power(true - pred, 2).mean())
mape = np.abs((true - pred) / true * 100).mean()
from sklearn.metrics import r2_score
r2 = r2_score(true, pred)

print(f"MAE : {mae:.3f} | RMSE : {rmse:.3f} | MAPE : {mape:.3f} | R2 : {r2:.3f}")

마지막으로 완성된 model를 저장하면서 마치도록 하겠습니다.

In [ ]:
PATH = "./my_better_cute_potato.model"
torch.save(model.state_dict(), PATH)

## Discussion

자, 이제 AttentiveFP model을 통해 얻어낸 결과들에 대해서 생각을 하나씩 해보도록 하겠습니다.

1. 계산 속도

확실히 epoch 하나 넘어가는데 NN보다는 오래 걸리죠? NN보다 훨씬 더 계산량이 훨씬 더 많으니까 당연하죠. 우리가 GPU를 이용하면서 model을 training 시키는데는 이런 이유들 때문입니다. GPU 사용환경이나 종류에 따라 다르겠지만 8 CPU < 1 GPU정도는 거의 확실하게 보장이 되니까요.

2. training 효율 (?)

이런 지표를 효율이라고 부르는지는 모르겠는데, 확실히 적은 epoch 숫자만으로 loss가 감소하는걸 볼 수 있죠. 아무래도 하나의 chemical compound를 나타내는 경우 graph가 더 rich한 정보들을 갖고, 이러한 rich한 정보를 바탕으로 model의 trainable parameter를 optimize하는데 더 많은 정보를 활용할 수 있겠습니다.

3. 정확도

사실 learning rate 조절도 안하고 early stopping조차도 적용 안한 결과들이라 결과에 대해서 이야기하는게 nonsense지만, 계산된 error metric을 NN의 결과와 비교해보면 큰 차이는 없는 것 같네요. 하지만 GNN의 loss curve를 보면 loss가 중간에 증가하는 부분이 보이죠? 이런 부분들을 learning rate를 잘 조절하면서 ([ReduceLROnPlateau](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html) 같은거 알아서 잘 공부해봐요) 학습시켰다면 model이 더 개선될 수 있는 여지는 충분한 것 같네요.

4. 개선 방안

뭐, 이부분은 더 이상 내가 할건 아니지만 아이디어는 몇개 던져볼게요. 관심 있는 사람은 (누가 있을런진 모르겠지만) 이 model을 더 개선시킬 수 있도록 노력해보아요.

  - learning rate scheduler
  - hyperparameter optimization - `batch_size`, `hidden_size`, `num_timesteps`, `num_layers`, etc.
  - early stopping 

## Outtro

이제 들꽃반을 시작하면서 "여기까지는 알려줘야겠다" 생각했던 부분이 마무리가 되었네요. 이 이상의 일들은 저도 새로 배워가야 하는 부분입니다. 그래도 지금까지 배워온 기초들이 나중에 여러 방향으로 사용되고 발전시킬 수 있음은 자신있게 얘기할 수 있습니다 (의지만 있다면요, 난 없지만 ㅎ). 제가 해줄 수 있는 역할들도 이제 마무리가 되는 것 같네요. 이 이후로도 Lecture가 더 추가될지는 잘 모르겠지만 여기서 한번 지금까지 수고 많았다는 인사를 던지도록 하겠습니다.